In [1]:

import pandas as pd
import os
import re
from pathlib import Path
from typing import Union

## Scenarios to analyse:
- ATCO006 - Scenario 1: 
    - Start: 1756973880000 (2025-09-04 08:26:40 UTC)
    - Stop: 1756974060000 (2025-09-04 08:29:40 UTC)
- ATCO001 - Scenario 3: 
    - Start: 1758890880000 (2025-09-26 12:48:00 UTC)
    - Stop: 1758891060000 (2025-09-26 12:51:00 UTC)
- ATCO007 - Scenario 2: 
    - Start: 1758091320000 (2025-09-17 06:26:40 UTC)
    - Stop: 1758091500000 (2025-09-17 06:29:40 UTC)

****
# Reproducing data loading pipeline (load_and_process_et)
****

WARNING: Reconstructing the chunks (windows) from scratch by just extracting the part that we want to analyse in the data is a bad idea because some task might began or end before or after the part we have doenloaded. Hw have to find a way to either:
- produce the prediction for the whole scenatio and then only select the part we want to display
- Load the chunks corresponding to the part we want to analyse only corresponding to the task we want to analyse. 

NB: The chunks per se are not saved. Only the training and test datasets. Either in those datasets the epoch is available to select those, or we have to use solution 1. 

NB: Solution 1 is longer but more straightforward.

In [165]:
from utils.task_data_io import PARQUET_ET_NAME, list_parquet_files
from utils.data_processing_gaze_data import EyeTrackingProcessor

# Loading full scenario because the prediction has to be made on the whole scenario to be relevant
def load_scenario(
    root: Union[str, Path],
    participant_id: Union[str, int],
    scenario_id: Union[str, int],
    # epoch_start: int,
    # epoch_end: int,
    timestamp_col: str = "epoch_ms", #(equal to epoch_ms_sync because preprocessed before)
):
    """
    Load one participant + scenario and filter rows between two epoch timestamps (epoch_ms).
    """
    
    # if epoch_end < epoch_start:
    #     raise ValueError("epoch_end must be >= epoch_start")
    
    # Find eye tracking and asd file names
    et_file_index, asd_file_index = list_parquet_files(root, participants=participant_id, scenarios=scenario_id)
    
    # Loading eye tracking data between epoch
    item = et_file_index[0]
    p_et = item["path"]
    df_et = pd.read_parquet(p_et)
    
    df_et = df_et.copy()
    # df_et= df_et[df_et[timestamp_col].between(epoch_start, epoch_end)]
    df_et["participant_id"] = str(item["participant_id"])
    df_et["scenario_id"] = str(item["scenario_id"])
    
    #loading asd events 
    item = asd_file_index[0]
    p_asd = item["path"]
    df_asd = pd.read_parquet(p_asd)
    
    df_asd = df_asd.copy()
    # df_asd= df_asd[df_asd["epoch_ms"].between(epoch_start, epoch_end)]
    df_asd["participant_id"] = str(item["participant_id"])
    df_asd["scenario_id"] = str(item["scenario_id"])

    return df_et, df_asd

store_dir = "/store/kruu/eye_tracking"
data_dir = os.path.join(store_dir, "training_data")    
df_et, df_asd = load_scenario(root = data_dir, 
                             participant_id = ["004"],
                             scenario_id = ["2"],)

In [166]:
# Task map defined when loading all data before training. Should keep the same in inference (could be useful to actually save it in the logs)

task_map = {'Aircraft requests': 'Task 0',
 'Assume': 'Task 1',
 'Conflict resolution': 'Task 2',
 'Entry conditions': 'Task 3',
 'Entry conflict resolution': 'Task 4',
 'Entry coordination': 'Task 5',
 'Exit conditions': 'Task 6',
 'Exit conflict resolution': 'Task 7',
 'Exit coordination': 'Task 8',
 'Non-conformance resolution': 'Task 9',
 'QoS': 'Task 10',
 'Return to route': 'Task 11',
 'Transfer': 'Task 12',
 'Zone conflict': 'Task 13'}

In [167]:
# Mimic the output after function "load data"

def apply_existing_task_map(
    df: pd.DataFrame,
    task_map: dict[str, str],
    event_col: str = "Event",
) -> pd.DataFrame:
    """
    Apply an existing task_map to a single dataframe.
    
    Converts:
        'Assume - start' → 'Task 1'
        'Assume - end'   → 'Task 1 end'
    
    Leaves unknown or non-task events unchanged.
    """

    df = df.copy()

    def map_event(event):
        if not isinstance(event, str) or " - " not in event:
            return event

        root, suffix = event.split(" - ", maxsplit=1)

        task_label = task_map.get(root)
        if task_label:
            if suffix.strip().lower() == "start":
                return task_label
            elif suffix.strip().lower() == "end":
                return f"{task_label} end"

        # If root not in task_map, return original
        return event

    df[event_col] = df[event_col].apply(map_event)

    return df

df_et_task = apply_existing_task_map(df_et, task_map)

In [168]:
columns = ['Recording timestamp [ms]', 'epoch_ms', 'Gaze point X [DACS px]', 'Gaze point Y [DACS px]', 'Event']

# Calculate the blinks
processor = EyeTrackingProcessor()
df_et_task_with_blinks, blink_summaries = processor.detect_blinks_in_streams([df_et_task])

# Get the multiscales windows: put the same parameters as training
chunks_et = processor.get_multiscale_window_chunks(
            df_et_task_with_blinks,
            features=columns + ["Blink", "Loss of Attention"],
            window_short_ms = 5000,
            window_mid_ms = 10000,
            window_long_ms = 25000,
            task_margin_ms=2000,
            step_ms=3000,
            filter_outliers=True,
        )

# NB: don't drop nans because we don't need to clean during inference

Finding tasks for participant 004 Scenario 2


****
# Feature extraction
****

In [169]:
# handmade features extraction
from tqdm import tqdm
from utils.data_processing_gaze_data import GazeMetricsProcessor
from utils.data_processing_mouse_data import MouseMetricsProcessor
from utils.data_processing_asd_events import ASDEventsMetricsProcessor

df_asd = df_asd.sort_values("epoch_ms").set_index("epoch_ms")

window_keys = ("short", "mid", "long")
rows = []

for uid, windows in tqdm(chunks_et.items()):
    
    row = {"id": uid}
    row["participant_id"] = windows["short"]["Participant name"].iloc[0]
    row["Task_id"] = windows["short"]["Task_id"].iloc[0]
        
    for wname in window_keys:
        chunk = windows[wname]
        min_epoch = chunk["epoch_ms"].min()
        max_epoch = chunk["epoch_ms"].max()
        
        window_asd = df_asd.loc[min_epoch:max_epoch].reset_index()
        
        # Gaze metrics
        gaze_processor = GazeMetricsProcessor(chunk, timestamp_unit="ms")
        gaze_metrics = gaze_processor.compute_all_metrics()   
        prefixed_gaze = {f"{wname}_{k}": v for k, v in gaze_metrics.items()}
        row.update(prefixed_gaze)
        
        # Mouse metrics
        mouse_processor = MouseMetricsProcessor(window_asd, resample=False)
        mouse_metrics = mouse_processor.compute_all_metrics()   
        prefixed_mouse = {f"{wname}_{k}": v for k, v in mouse_metrics.items()}
        row.update(prefixed_mouse)
        
        # ASD events metrics
        asd_processor = ASDEventsMetricsProcessor(window_asd)
        asd_metrics = asd_processor.compute_all_metrics()   
        prefixed_asd = {f"{wname}_{k}": v.iat[0] for k, v in asd_metrics.items()}
        row.update(prefixed_asd)
    
    rows.append(row)

metrics_df = pd.DataFrame(rows)

100%|██████████| 1137/1137 [04:36<00:00,  4.11it/s]


In [170]:
import joblib
from trainings._01_xgboost_hierarchical_training import extract_tsfresh_features_from_multiscale_chunks

def tsfresh_for_model_features(
    multiscale_chunks: dict[str, dict[str, pd.DataFrame]],
    columns_to_extract: list[str],
    n_jobs: int = 50,
) -> pd.DataFrame:
    """
    1) Computes TSFresh features WITHOUT thresholding (keeps everything produced)
    Returns a dataframe with columns: ['id', <tsfresh cols that model expects>]
    """

    # ---- compute ALL tsfresh features for all scales (no thresholding) ----
    tsfresh_df = extract_tsfresh_features_from_multiscale_chunks(
        multiscale_chunks=multiscale_chunks,
        columns_to_extract=columns_to_extract,
        pval_threshold=1,   # IMPORTANT: no selection at inference
        n_jobs=n_jobs,
    )

    return tsfresh_df

tsfresh_data = tsfresh_for_model_features(
    multiscale_chunks=chunks_et,
    columns_to_extract=['Gaze point X [DACS px]', 'Gaze point Y [DACS px]'],
)

Extracting TSFresh features...


Feature Extraction: 100%|██████████| 228/228 [00:00<00:00, 318.86it/s]


Extracting TSFresh features...


Feature Extraction: 100%|██████████| 228/228 [00:00<00:00, 278.32it/s]


Extracting TSFresh features...


Feature Extraction: 100%|██████████| 228/228 [00:01<00:00, 204.95it/s]


In [171]:
from trainings._01_xgboost_hierarchical_training import build_xy

# Merging data
merged_data = metrics_df.merge(tsfresh_data, on="id", how="inner")

# Building XGBoost input df and sanitzing column names (within build_xy)
# We have the ground truth here if needed. 
X, y_tasks, y_active, _ = build_xy(merged_data, [])

# Drop features that are not in model features (because of pval thresholding in tsfresh)
bundle_A = joblib.load("trainings/logs/xgboost_hierarchical_v5/best_model_stageA.pkl")
model_A = bundle_A["model"]
expected = list(model_A.feature_names_in_)
X = X.reindex(columns=expected)

# Ensure numeric
for c in X.columns:
    if not pd.api.types.is_numeric_dtype(X[c]):
        X[c] = pd.to_numeric(X[c], errors="coerce").fillna(0.0)

In [172]:
X.head(10)

,short_Fixation_Count,short_Total_Fixation_Duration_(s),short_Avg_Fixation_Duration_(s),short_Saccade_Count,short_Avg_Saccade_Amplitude_(px),short_Avg_Saccade_Velocity_(px/s),short_Avg_Gaze_Velocity_(px/s),short_Avg_Gaze_Acceleration_(px/s²),short_Blink_Rate_(blinks/s),short_Gaze_Dispersion_(area_px²),...,long_Gaze_point_Y__DACS_px___mean,long_Gaze_point_X__DACS_px___mean,long_Gaze_point_Y__DACS_px___sum_values,long_Gaze_point_X__DACS_px___sum_values,long_Gaze_point_X__DACS_px___absolute_maximum,long_Gaze_point_X__DACS_px___maximum,long_Gaze_point_Y__DACS_px___standard_deviation,long_Gaze_point_Y__DACS_px___variance,long_Gaze_point_Y__DACS_px___median,long_Gaze_point_X__DACS_px___root_mean_square
0,8.0,1.488,0.186000,36.0,143.641785,17429.380859,1168.639526,-55758.335938,15.828491,1997230.0,...,772.519836,1051.917725,2722360.0,3706958.0,2491.0,2491.0,349.586426,122210.671875,944.5,1107.270386
1,11.0,2.232,0.202909,20.0,129.236679,15658.899414,871.639648,-10710.545898,36.666000,1667640.0,...,745.990112,1151.667969,2637821.0,4072298.0,2491.0,2491.0,369.531250,136553.328125,729.0,1223.342529
2,13.0,2.728,0.209846,6.0,308.294312,8474.064453,1132.199951,-20155.160156,29.246796,399840.0,...,691.092163,1056.357666,2451995.0,3747957.0,2491.0,2491.0,353.687866,125095.101562,649.0,1131.962769
3,7.0,1.600,0.228571,4.0,325.256470,8751.455078,642.844666,349.047211,68.910255,355641.0,...,469.106110,948.051331,1626860.0,3287842.0,2491.0,2491.0,266.698975,71128.335938,555.0,1012.763062
4,2.0,0.296,0.148000,1.0,290.027588,9073.727539,138.041931,136.841660,109.174683,38398.0,...,413.009430,905.337463,1400102.0,3069094.0,2491.0,2491.0,294.847443,86935.007812,418.0,971.843140
5,13.0,2.640,0.203077,10.0,292.393738,8210.212891,1201.448486,1084.335571,36.858974,445051.0,...,428.229462,868.095459,1422150.0,2882945.0,2491.0,2491.0,295.866364,87536.898438,473.0,928.066040
6,19.0,3.616,0.190316,10.0,431.603790,10180.492188,1906.487793,1280.259155,5.208333,864558.0,...,435.934662,844.981567,1421147.0,2754640.0,2491.0,2491.0,307.459595,94531.398438,463.0,908.177368
7,12.0,2.344,0.195333,8.0,395.607025,11832.386719,1266.606567,-16615.986328,39.463142,821988.0,...,407.039948,819.892456,1294387.0,2607258.0,2491.0,2491.0,277.905457,77231.437500,404.0,879.229553
8,17.0,3.592,0.211294,7.0,252.175003,9834.392578,1172.904907,-17927.996094,10.216346,301889.0,...,428.057800,802.291565,1318418.0,2471058.0,1621.0,1621.0,276.370392,76380.593750,524.0,846.557251
9,13.0,3.384,0.260308,7.0,213.890121,6774.963867,888.595459,330.930115,28.645834,199533.0,...,421.771881,800.712769,1290622.0,2450181.0,1446.0,1446.0,278.543976,77586.742188,535.0,828.730957


In [173]:
# TESTING FEATURES MATCHING

import numpy as np

expected = np.array(model_A.feature_names_in_, dtype=str)
got      = np.array(X.columns, dtype=str)

missing_in_X = sorted(set(expected) - set(got))
extra_in_X   = sorted(set(got) - set(expected))

print(f"Expected features (model): {len(expected)}")
print(f"Got features (X):          {len(got)}")
print(f"Missing in X:              {len(missing_in_X)}")
print(f"Extra in X:                {len(extra_in_X)}")

if missing_in_X:
    print("\n--- First 50 missing ---")
    print("\n".join(missing_in_X[:50]))

if extra_in_X:
    print("\n--- First 50 extra ---")
    print("\n".join(extra_in_X[:50]))

# Optional: check ordering of common columns
common = [c for c in expected if c in set(got)]
wrong_order = sum(got.tolist().index(c) != i for i, c in enumerate(common))
print(f"\nCommon features:           {len(common)}")
print(f"Common but wrong position: {wrong_order} (relative to model order)")

Expected features (model): 459
Got features (X):          459
Missing in X:              0
Extra in X:                0

Common features:           459
Common but wrong position: 0 (relative to model order)


****
# Model inference
****

In [174]:
import numpy as np
import joblib
from trainings._01_xgboost_hierarchical_training import build_xy

def infer_hierarchical_smoothed(
  X: pd.DataFrame,
  model_dir: Union[str, Path],
  idle_label: int = -1,          # same as training
  alpha_B: float = 0.6,         # same as training report (though it could be changed)
  ) -> tuple[pd.Series, pd.DataFrame]:
  """
  Returns:
    - yhat: predicted class label per row (index aligned to xgboost_data)
    - P:    combined (smoothed) class probabilities dataframe
  """

  model_dir = Path(model_dir)

  # Load model stages
  bundle_A = joblib.load(model_dir / "best_model_stageA.pkl")
  model_A = bundle_A["model"]
  threshold_A = bundle_A.get("threshold", None)

  bundle_B = joblib.load(model_dir / "best_model_stageB.pkl")
  model_B = bundle_B["model"]

  # Stage A probabilities (Active VS idle)
  pA = pd.Series(model_A.predict_proba(X)[:, 1], index=X.index, name="p_active")

  # Stage B probabilities (Task id)
  prob_B = model_B.predict_proba(X)
  classes_B = model_B.classes_  # labels for columns in prob_B
  active_classes = np.array(classes_B)
  pB = pd.DataFrame(prob_B, index=X.index, columns=active_classes)
  
  # smooth Stage B (EMA over time)
  pB_smooth = pB.ewm(alpha=alpha_B, adjust=False).mean()
  # We will need to adapt the smoothing when predicting in real-time. We will need to store the previous predictions
  # Here previous predictions are already stored as we predict a batch on consecutive times at once. 

  # rebuild combined probabilities over ALL classes (include IDLE)
  all_cols = [idle_label] + list(active_classes)
  P = pd.DataFrame(0.0, index=X.index, columns=all_cols)

  P[idle_label] = 1.0 - pA.values
  for c in active_classes:
      P[c] = pA.values * pB_smooth[c].values

  # (optional) renormalize to sum to 1
  row_sum = P.sum(axis=1).replace(0.0, 1.0)
  P = P.div(row_sum, axis=0)

  # final prediction
  yhat = P.idxmax(axis=1)

  return yhat, P

In [175]:
y_hat, P = infer_hierarchical_smoothed(
    X=X,
    model_dir="trainings/logs/xgboost_hierarchical_v5",
    idle_label=-1,
    alpha_B=0.6,
)

In [176]:
ids = []
epochs = []
for _, dic in chunks_et.items(): 
    ids.append(dic["short"]["id"].min())
    epochs.append(dic["short"]["epoch_ms"].max().item())

out = pd.DataFrame({"id": ids, "prediction_epoch_ms": epochs})

out["pred_task"] = np.asarray(y_hat)
out["true_task"] = np.asarray(y_tasks)
out["pred_proba"] = P.to_numpy().max(axis=1)


In [177]:
# import pandas as pd

# def best_2min_window_select_on_active_only(
#     df: pd.DataFrame,
#     ts_col="prediction_epoch_ms",
#     pred_col="pred_task",
#     true_col="true_task",
#     idle=-1,
#     window_ms=2*60*1000,
# ):
#     """
#     Select best 2-min window by accuracy computed ONLY on rows with true_task != idle.
#     """
#     d_all = df.copy().sort_values(ts_col).reset_index(drop=True)

#     # rows used for scoring (active-only by TRUE label)
#     d_score = d_all[d_all[true_col] != idle].copy().reset_index(drop=True)
#     if d_score.empty:
#         return None, None, None

#     d_score["correct"] = (d_score[pred_col] == d_score[true_col]).astype(int)

#     t = d_score[ts_col].to_numpy()
#     c = d_score["correct"].to_numpy()

#     best = {"acc": -1.0, "start": None, "end": None, "n": 0, "correct": 0}
#     j = 0
#     running_correct = 0

#     for i in range(len(d_score)):
#         while j < len(d_score) and t[j] < t[i] + window_ms:
#             running_correct += c[j]
#             j += 1

#         n = j - i
#         if n > 0:
#             acc = running_correct / n
#             if (acc > best["acc"]) or (acc == best["acc"] and n > best["n"]):
#                 best.update({
#                     "acc": acc,
#                     "start": int(t[i]),
#                     "end": int(t[i] + window_ms),
#                     "n": int(n),
#                     "correct": int(running_correct),
#                 })

#         running_correct -= c[i]

#     # ALL rows in that time window (no filtering)
#     best_window_rows = d_all[
#         (d_all[ts_col] >= best["start"]) & (d_all[ts_col] < best["end"])
#     ].copy()

#     summary = pd.DataFrame([{
#         "window_start_epoch_ms": best["start"],
#         "window_end_epoch_ms": best["end"],
#         "n_active_scored": best["n"],
#         "n_active_correct": best["correct"],
#         "active_accuracy": best["acc"],
#         "n_rows_total_in_window": len(best_window_rows),
#         "n_idle_true_in_window": int((best_window_rows[true_col] == idle).sum()),
#     }])

#     return summary, best_window_rows

# summary, best_window_rows = best_2min_window_select_on_active_only(out)
# print(summary)

In [178]:
import pandas as pd

def best_2min_window_by_correct_active(
    df: pd.DataFrame,
    ts_col="prediction_epoch_ms",
    pred_col="pred_task",
    true_col="true_task",
    idle=-1,
    window_ms=2*60*1000,
    min_active: int = 10,         
    acc_tiebreak: float = 0.3,     
):
    d_all = df.copy().sort_values(ts_col).reset_index(drop=True)

    d_score = d_all[d_all[true_col] != idle].copy().reset_index(drop=True)
    if d_score.empty:
        return None, None

    d_score["correct"] = (d_score[pred_col] == d_score[true_col]).astype(int)

    t = d_score[ts_col].to_numpy()
    c = d_score["correct"].to_numpy()

    best = {"correct": -1, "acc": -1.0, "start": None, "end": None, "n": 0}
    j = 0
    running_correct = 0

    for i in range(len(d_score)):
        while j < len(d_score) and t[j] < t[i] + window_ms:
            running_correct += c[j]
            j += 1

        n = j - i
        if n >= min_active:
            acc = running_correct / n
            if acc >= acc_tiebreak:
                # primary: maximize correct; secondary: maximize accuracy; then maximize n
                if (running_correct > best["correct"]) or \
                   (running_correct == best["correct"] and acc > best["acc"]) or \
                   (running_correct == best["correct"] and acc == best["acc"] and n > best["n"]):
                    best.update({
                        "correct": int(running_correct),
                        "acc": float(acc),
                        "start": int(t[i]),
                        "end": int(t[i] + window_ms),
                        "n": int(n),
                    })

        running_correct -= c[i]

    if best["start"] is None:
        return None, None  # no window met min_active / acc_tiebreak

    best_window_rows = d_all[(d_all[ts_col] >= best["start"]) & (d_all[ts_col] < best["end"])].copy()

    summary = pd.DataFrame([{
        "window_start_epoch_ms": best["start"],
        "window_end_epoch_ms": best["end"],
        "n_active_scored": best["n"],
        "n_active_correct": best["correct"],
        "active_accuracy": best["acc"],
        "n_rows_total_in_window": len(best_window_rows),
    }])

    return summary, best_window_rows

summary, best_window_rows = best_2min_window_by_correct_active(out)
print(summary)

   window_start_epoch_ms  window_end_epoch_ms  n_active_scored  \
0          1757689424621        1757689544621               17   

   n_active_correct  active_accuracy  n_rows_total_in_window  
0                16         0.941176                      41  


In [179]:
best_window_rows

,id,prediction_epoch_ms,pred_task,true_task,pred_proba
1049,004_2_8_10,1757689424621,8,8,0.830335
1050,004_2_8_11,1757689427621,8,8,0.791719
1051,004_2_8_12,1757689430621,8,8,0.818702
1052,004_2_8_13,1757689433620,8,8,0.833975
1053,004_2_8_14,1757689436620,8,8,0.838461
1054,004_2_8_15,1757689439620,8,8,0.839191
1055,004_2_8_16,1757689442618,8,8,0.860665
1056,004_2_8_17,1757689445618,8,8,0.892401
1057,004_2_8_18,1757689448618,8,8,0.762023
1058,004_2_8_19,1757689451618,8,8,0.832284


# Cherry picked 2-minute slots:
- ATCO001 - Scenario 3: 1758889892503 - 1758890009515 (accuracy: 0.56 - 17 active correct) very diverse
- ATCO001 - Scenario 1: 1758700993572 - 1758701110584 (accuracy: 0.78 - 7 active correct)
- ATCO002 - Scenario 2: 1757489247109 - 1757489364112 (accuracy: 0.73 - 11 active correct)
- ATCO002 - Scenario 3: 1757600815852 - 1757600932860 (accuracy: 0.81 - 9 active correct)
- ATCO003 - Scenario 1: 1758784318946 - 1758784435959 (accuracy: 0.91 - 21 active correct) very interesting as well
- ATCO003 - Scenario 3: 1758705557261 - 1758705674282 (accuracy: 0.97 - 30 active correct) very interesting
- ATCO004 - Scenario 1: 1757493758065 - 1757493875102 (accuracy: 0.79 - 19 active correct)
- ATCO004 - Scenario 2: 1757689424621 - 1757689544607 (accuracy: 0.94 - 16 active correct) very interesting